In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import skew
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import statsmodels.api as sm

# Tell sklearn to output Pandas DataFrames from transformers
# This is crucial for CatBoost's native handling
import sklearn
sklearn.set_config(transform_output="pandas")

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("All libraries imported successfully.")

All libraries imported successfully.


In [7]:
# Load the dataset
file_path = 'merged_txn_data.csv'
try:
    data = pd.read_csv(file_path)
    print(f"Dataset loaded successfully: {file_path}")
    print(f"Original shape: {data.shape}")
except FileNotFoundError:
    print(f"ERROR: File not found at {file_path}")
    # Handle error appropriately

Dataset loaded successfully: merged_txn_data.csv
Original shape: (5844, 23)


In [8]:
# --- Feature Engineering & Cleaning (from your notebook) ---
# We use the exact same cleaning steps to ensure a fair comparison.

# Drop columns that are irrelevant or have high multicollinearity
columns_to_drop = [
    'main_district', 
    'price_per_sqft', 
    'easting', 
    'northing', 
    'distance_to_nearest_mtr_km'
]
data = data.drop(columns=columns_to_drop)

# Handle outliers
def handle_outliers(df, column, threshold=0.01):
    low_percentile = df[column].quantile(threshold)
    high_percentile = df[column].quantile(1 - threshold)
    
    # Cap and floor the outliers
    df[column] = np.clip(df[column], low_percentile, high_percentile)
    return df

data = handle_outliers(data, 'saleable_area', threshold=0.01)
data = handle_outliers(data, 'property_age', threshold=0.01)

# Drop rows with NaN in 'price' (target variable)
data.dropna(subset=['price'], inplace=True)

print(f"Data shape after cleaning: {data.shape}")

Data shape after cleaning: (5844, 18)


In [9]:
# --- Define Feature Lists for A/B Comparison ---

# These are the 3 POI features you want to test
poi_features_to_test = [
    'total_poi_within_1000m',
    'category_Education_within_2000m',
    'category_Medical_within_2000m'
]

# These are the *other* numerical features we will always include
numerical_base_features = [
    'saleable_area',
    'latitude',
    'longitude',
    'bedroom_count',
    'property_age',
    'travel_time_to_cbd',
    'walking_time_to_mtr'
]

# This is the categorical feature we will always include
categorical_features = [
    'housing_market_area',
]

# --- Define feature lists for our TWO experiments ---

# 1. Standard Features (No PCA)
# We combine the base numerical features + the 3 POI features
numerical_cols_standard = numerical_base_features + poi_features_to_test
categorical_cols_standard = categorical_features

# 2. PCA Features
# The list of features for PCA is defined separately
# The base numerical features are defined separately
# The categorical features are the same
numerical_cols_pca_base = numerical_base_features
poi_cols_for_pca = poi_features_to_test
categorical_cols_pca = categorical_features

print("Feature lists defined for Standard and PCA runs.")

Feature lists defined for Standard and PCA runs.


In [10]:
# --- Split the Data ---
# Use all columns defined so we don't have to split twice
# This ensures both runs use the exact same test set
all_features = list(set(numerical_cols_standard + categorical_cols_standard))
y = data['price']
X = data[all_features]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Data split into training ({X_train.shape}) and testing ({X_test.shape}) sets.")

Data split into training ((4675, 11)) and testing ((1169, 11)) sets.


In [11]:
# --- Evaluation Function ---
# This function calculates R2, RMSE, MAE, MAPE, and Accuracy Ranges

def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100 # Show as percentage
    
    # Calculate accuracy ranges
    error_pct = np.abs((y_test - y_pred) / y_test)
    acc_10 = np.mean(error_pct <= 0.1) * 100
    acc_20 = np.mean(error_pct <= 0.2) * 100
    acc_30 = np.mean(error_pct <= 0.3) * 100
    
    print(f"\n--- Results for {model_name} ---")
    print(f"R-squared: {r2:.4f}")
    print(f"RMSE: {rmse:,.2f}")
    print(f"MAE: {mae:,.2f}")
    print(f"MAPE: {mape:.2f}%")
    print(f"Accuracy (within 10%): {acc_10:.2f}%")
    print(f"Accuracy (within 20%): {acc_20:.2f}%")
    print(f"Accuracy (within 30%): {acc_30:.2f}%")
    
    return {
        'Model': model_name,
        'R-squared': r2,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'Accuracy (10%)': acc_10,
        'Accuracy (20%)': acc_20,
        'Accuracy (30%)': acc_30
    }

In [12]:
# --- Define Preprocessing Pipelines ---

# --- RUN 1: STANDARD PREPROCESSORS (No PCA) ---

# 1. Standard Preprocessor (for XGB, LGBM)
num_transformer_standard = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer_standard = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor_standard = ColumnTransformer(
    transformers=[
        ('num', num_transformer_standard, numerical_cols_standard),
        ('cat', cat_transformer_standard, categorical_cols_standard)
    ],
    verbose_feature_names_out=False
)

# 2. Standard Preprocessor (for CatBoost)
preprocessor_catboost_standard = ColumnTransformer(
    transformers=[
        ('num', num_transformer_standard, numerical_cols_standard),
        ('cat', 'passthrough', categorical_cols_standard) # <-- Pass-through
    ],
    verbose_feature_names_out=False
)

print("Standard Preprocessors (no PCA) created.")


# --- RUN 2: PCA PREPROCESSORS ---

# 1. PCA Preprocessor (for XGB, LGBM)
# Pipeline for the 3 POI features: Impute -> Scale -> PCA
poi_pca_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95, random_state=42)) # Keeps 95% of variance
])
# Pipeline for the other numerical features: Impute -> Scale
num_base_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
# Categorical pipeline is the same
cat_pca_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor_pca = ColumnTransformer(
    transformers=[
        ('poi_pca', poi_pca_pipeline, poi_cols_for_pca),
        ('num_base', num_base_pipeline, numerical_cols_pca_base),
        ('cat', cat_pca_pipeline, categorical_cols_pca)
    ],
    verbose_feature_names_out=False
)

# 2. PCA Preprocessor (for CatBoost)
preprocessor_catboost_pca = ColumnTransformer(
    transformers=[
        ('poi_pca', poi_pca_pipeline, poi_cols_for_pca),
        ('num_base', num_base_pipeline, numerical_cols_pca_base),
        ('cat', 'passthrough', categorical_cols_pca) # <-- Pass-through
    ],
    verbose_feature_names_out=False
)

print("PCA Preprocessors created.")

Standard Preprocessors (no PCA) created.
PCA Preprocessors created.


In [13]:
# --- Model Training (Run 1 - Standard Features) ---

print("="*50)
print("STARTING RUN 1: STANDARD FEATURES")
print("="*50)

model_results = [] # To store all metrics

# --- XGBoost (Standard) ---
pipeline_xgb_std = Pipeline(steps=[
    ('preprocessor', preprocessor_standard),
    ('model', xgb.XGBRegressor(random_state=42, n_jobs=-1))
])
params_xgb = {'model__n_estimators': [500], 'model__learning_rate': [0.1], 'model__max_depth': [8]}
grid_xgb_std = GridSearchCV(pipeline_xgb_std, params_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_xgb_std.fit(X_train, y_train)
model_results.append(evaluate_model(grid_xgb_std.best_estimator_, X_test, y_test, 'XGBoost_Standard'))

# --- LightGBM (Standard) ---
pipeline_lgbm_std = Pipeline(steps=[
    ('preprocessor', preprocessor_standard),
    ('model', lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1))
])
params_lgbm = {'model__n_estimators': [500], 'model__learning_rate': [0.1], 'model__num_leaves': [50]}
grid_lgbm_std = GridSearchCV(pipeline_lgbm_std, params_lgbm, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_lgbm_std.fit(X_train, y_train)
model_results.append(evaluate_model(grid_lgbm_std.best_estimator_, X_test, y_test, 'LightGBM_Standard'))

# --- CatBoost (Standard) ---
pipeline_cb_std = Pipeline(steps=[
    ('preprocessor', preprocessor_catboost_standard),
    ('model', CatBoostRegressor(random_state=42, verbose=0))
])
params_cb = {'model__iterations': [1000], 'model__learning_rate': [0.1], 'model__depth': [8]}
grid_cb_std = GridSearchCV(pipeline_cb_std, params_cb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

# CatBoost needs categorical feature names passed to .fit()
X_train_cb = X_train.copy()
for col in categorical_cols_standard:
    X_train_cb[col] = X_train_cb[col].astype(str)

grid_cb_std.fit(X_train_cb, y_train, model__cat_features=categorical_cols_standard)
model_results.append(evaluate_model(grid_cb_std.best_estimator_, X_test, y_test, 'CatBoost_Standard'))

STARTING RUN 1: STANDARD FEATURES
Fitting 5 folds for each of 1 candidates, totalling 5 fits

--- Results for XGBoost_Standard ---
R-squared: 0.8802
RMSE: 1,812,341.29
MAE: 1,049,953.50
MAPE: 11.20%
Accuracy (within 10%): 57.31%
Accuracy (within 20%): 85.80%
Accuracy (within 30%): 93.76%
Fitting 5 folds for each of 1 candidates, totalling 5 fits

--- Results for LightGBM_Standard ---
R-squared: 0.8766
RMSE: 1,839,249.73
MAE: 1,061,613.32
MAPE: 11.20%
Accuracy (within 10%): 58.25%
Accuracy (within 20%): 85.12%
Accuracy (within 30%): 94.44%
Fitting 5 folds for each of 1 candidates, totalling 5 fits

--- Results for CatBoost_Standard ---
R-squared: 0.8897
RMSE: 1,739,045.88
MAE: 1,015,752.36
MAPE: 10.80%
Accuracy (within 10%): 58.60%
Accuracy (within 20%): 86.66%
Accuracy (within 30%): 94.95%


In [14]:
# --- Model Training (Run 2 - PCA Features) ---

print("="*50)
print("STARTING RUN 2: PCA FEATURES")
print("="*50)

# We append to the *same* model_results list

# --- XGBoost (PCA) ---
pipeline_xgb_pca = Pipeline(steps=[
    ('preprocessor', preprocessor_pca),
    ('model', xgb.XGBRegressor(random_state=42, n_jobs=-1))
])
# We use the *exact same* model parameters
grid_xgb_pca = GridSearchCV(pipeline_xgb_pca, params_xgb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_xgb_pca.fit(X_train, y_train)
model_results.append(evaluate_model(grid_xgb_pca.best_estimator_, X_test, y_test, 'XGBoost_PCA'))

# --- LightGBM (PCA) ---
pipeline_lgbm_pca = Pipeline(steps=[
    ('preprocessor', preprocessor_pca),
    ('model', lgb.LGBMRegressor(random_state=42, n_jobs=-1, verbose=-1))
])
grid_lgbm_pca = GridSearchCV(pipeline_lgbm_pca, params_lgbm, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
grid_lgbm_pca.fit(X_train, y_train)
model_results.append(evaluate_model(grid_lgbm_pca.best_estimator_, X_test, y_test, 'LightGBM_PCA'))

# --- CatBoost (PCA) ---
pipeline_cb_pca = Pipeline(steps=[
    ('preprocessor', preprocessor_catboost_pca),
    ('model', CatBoostRegressor(random_state=42, verbose=0))
])
grid_cb_pca = GridSearchCV(pipeline_cb_pca, params_cb, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

# We use the same prepared X_train_cb from the previous cell
grid_cb_pca.fit(X_train_cb, y_train, model__cat_features=categorical_cols_pca)
model_results.append(evaluate_model(grid_cb_pca.best_estimator_, X_test, y_test, 'CatBoost_PCA'))

STARTING RUN 2: PCA FEATURES
Fitting 5 folds for each of 1 candidates, totalling 5 fits

--- Results for XGBoost_PCA ---
R-squared: 0.8818
RMSE: 1,800,332.88
MAE: 1,048,704.75
MAPE: 11.19%
Accuracy (within 10%): 56.80%
Accuracy (within 20%): 84.94%
Accuracy (within 30%): 94.53%
Fitting 5 folds for each of 1 candidates, totalling 5 fits

--- Results for LightGBM_PCA ---
R-squared: 0.8786
RMSE: 1,824,569.45
MAE: 1,058,096.02
MAPE: 11.33%
Accuracy (within 10%): 57.23%
Accuracy (within 20%): 85.03%
Accuracy (within 30%): 94.10%
Fitting 5 folds for each of 1 candidates, totalling 5 fits

--- Results for CatBoost_PCA ---
R-squared: 0.8870
RMSE: 1,760,469.34
MAE: 1,036,478.73
MAPE: 11.09%
Accuracy (within 10%): 57.06%
Accuracy (within 20%): 85.71%
Accuracy (within 30%): 94.95%


In [15]:
# --- Final Comparison ---

# Convert the list of dictionaries into a DataFrame
results_df = pd.DataFrame(model_results)

# Sort by R-squared to see the best performing model
results_df = results_df.sort_values(by='R-squared', ascending=False)

# Display the final comparison table
print("\n" + "="*80)
print("FINAL MODEL COMPARISON (STANDARD vs. PCA)")
print("="*80)
print(results_df.to_markdown(index=False, floatfmt=".4f"))

# Save the comparison to a CSV file
results_df.to_csv("model_comparison_standard_vs_pca.csv", index=False)
print("\nComparison table saved to 'model_comparison_standard_vs_pca.csv'")


FINAL MODEL COMPARISON (STANDARD vs. PCA)
| Model             |   R-squared |         RMSE |          MAE |    MAPE |   Accuracy (10%) |   Accuracy (20%) |   Accuracy (30%) |
|:------------------|------------:|-------------:|-------------:|--------:|-----------------:|-----------------:|-----------------:|
| CatBoost_Standard |      0.8897 | 1739045.8815 | 1015752.3645 | 10.7956 |          58.5971 |          86.6553 |          94.9530 |
| CatBoost_PCA      |      0.8870 | 1760469.3430 | 1036478.7255 | 11.0883 |          57.0573 |          85.7143 |          94.9530 |
| XGBoost_PCA       |      0.8818 | 1800332.8819 | 1048704.7500 | 11.1910 |          56.8007 |          84.9444 |          94.5252 |
| XGBoost_Standard  |      0.8802 | 1812341.2919 | 1049953.5000 | 11.2026 |          57.3139 |          85.7998 |          93.7553 |
| LightGBM_PCA      |      0.8786 | 1824569.4467 | 1058096.0154 | 11.3348 |          57.2284 |          85.0299 |          94.0975 |
| LightGBM_Standard |     